In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict
from matplotlib.dates import AutoDateLocator, ConciseDateFormatter
from scipy.stats import gaussian_kde, norm
from collections import Counter
plt.rcParams.update({'figure.dpi': 400})
os.makedirs('../imgs',exist_ok=True)

# Load data

In [ ]:
train = pd.read_csv('../dataset/Train.csv')
test = pd.read_csv('../dataset/Test.csv')

In [ ]:
train.describe()

In [ ]:
test.describe()

# Null data

In [ ]:
def get_null_report(df):
    groups = defaultdict(list)
    for c in df.columns:
        if "_" in c:
            groups[c.split("_", 1)[0]].append(c)
    prefixes_to_check = [
        "sulphurdioxide","carbonmonoxide","nitrogendioxide","formaldehyde",
        "uvaerosolindex","ozone","uvaerosollayerheight","cloud"
    ]
    groups = {k:v for k,v in groups.items() if k in prefixes_to_check and len(v) >= 2}

    # Check consistency of missing values in each group
    report_rows = []
    bad_row_idx = set()

    for pref, cols in groups.items():
        na_cnt = df[cols].isna().sum(axis=1)       
        ncols = len(cols)
        partial = (na_cnt > 0) & (na_cnt < ncols)      # partially missing
        any_na = na_cnt > 0
        all_na = na_cnt == ncols

        bad_row_idx.update(df.index[partial])

        report_rows.append({
            "prefix": pref,
            "n_cols": ncols,
            "rows_total": len(df),
            "rows_any_NA": int(any_na.sum()),
            "rows_all_NA": int(all_na.sum()),
            "rows_partial_NA": int(partial.sum()),
            "OK_all_or_nothing": bool(partial.sum() == 0)
        })

    report = pd.DataFrame(report_rows).sort_values("prefix")
    return report

In [ ]:
report_train, report_test = get_null_report(train), get_null_report(test) #print will get all rows with same prefix full or nothing

In [ ]:
report_train = report_train.sort_values('rows_all_NA', ascending=False)
prefixes = report_train['prefix'].tolist()
x = np.arange(len(prefixes))
width = 0.28


y_all = report_train['rows_all_NA'].to_numpy()

tot = report_train['rows_total'].to_numpy()

# hbar plot with percentage labels
plt.figure(figsize=(9, 5))
plt.barh(prefixes, y_all)
for i, v in enumerate(y_all):
    pct = 100.0 * v / tot[i]
    plt.text(v, i, f' {v} ({pct:.1f}%)', va='center')
plt.xlabel('Number of rows')
plt.title('Percentage of completely missing rows by prefix - Train')
plt.tight_layout()
plt.savefig('../imgs/missing_rows_train.png')
plt.savefig('../imgs/missing_rows_train.pdf')
plt.show()


In [ ]:
report_test = report_test.sort_values('rows_all_NA', ascending=False)
prefixes = report_test['prefix'].tolist()
x = np.arange(len(prefixes))
width = 0.28


y_all = report_test['rows_all_NA'].to_numpy()

tot = report_test['rows_total'].to_numpy()

# hbar plot with percentage labels
plt.figure(figsize=(9, 5))
plt.barh(prefixes, y_all)
for i, v in enumerate(y_all):
    pct = 100.0 * v / tot[i]
    plt.text(v, i, f' {v} ({pct:.1f}%)', va='center')
plt.xlabel('Number of rows')
plt.title('Percentage of completely missing rows by prefix - Test')
plt.tight_layout()
plt.savefig('../imgs/missing_rows_test.png')
plt.savefig('../imgs/missing_rows_test.pdf')
plt.show()


# Analysis of Metadata

In [ ]:
train_stats = train.groupby('city').agg(
    num_sites=('site_id', 'nunique'),
    num_samples=('site_id', 'count')
).reset_index()

test_stats = test.groupby('city').agg(
    num_sites=('site_id', 'nunique'),
    num_samples=('site_id', 'count')
).reset_index()

max_site = max(train_stats['num_sites'].max(), test_stats['num_sites'].max()) + 2
max_sample = max(train_stats['num_samples'].max(), test_stats['num_samples'].max()) + 400

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), sharey=False)
width = 0.4

# TRAIN
x1 = np.arange(len(train_stats))
bars1_sites = ax1.bar(x1 - width/2, train_stats['num_sites'], width, color='dodgerblue', label='Sites')
ax1.set_ylabel('Number of Sites', color='dodgerblue')
ax1.tick_params(axis='y', labelcolor='dodgerblue')
ax1.set_ylim(0, max_site)

ax1b = ax1.twinx()
bars1_samples = ax1b.bar(x1 + width/2, train_stats['num_samples'], width, color='tomato', label='Samples')
ax1b.set_ylabel('Number of Samples', color='tomato')
ax1b.tick_params(axis='y', labelcolor='tomato')
ax1b.set_ylim(0, max_sample)

ax1.set_xticks(x1)
ax1.set_xticklabels(train_stats['city'], rotation=45)
ax1.set_title('Train Data')

for bar in bars1_sites:
    height = bar.get_height()
    ax1.annotate(f'{height}', xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3), textcoords="offset points", ha='center', fontsize=9, color='dodgerblue')

for bar in bars1_samples:
    height = bar.get_height()
    ax1b.annotate(f'{height}', xy=(bar.get_x() + bar.get_width() / 2, height),
                  xytext=(0, 3), textcoords="offset points", ha='center', fontsize=9, color='tomato')

# TEST
x2 = np.arange(len(test_stats))
bars2_sites = ax2.bar(x2 - width/2, test_stats['num_sites'], width, color='royalblue', label='Sites')
ax2.set_ylabel('Number of Sites', color='royalblue')
ax2.tick_params(axis='y', labelcolor='royalblue')
ax2.set_ylim(0, max_site)

ax2b = ax2.twinx()
bars2_samples = ax2b.bar(x2 + width/2, test_stats['num_samples'], width, color='firebrick', label='Samples')
ax2b.set_ylabel('Number of Samples', color='firebrick')
ax2b.tick_params(axis='y', labelcolor='firebrick')
ax2b.set_ylim(0, max_sample)

ax2.set_xticks(x2)
ax2.set_xticklabels(test_stats['city'], rotation=45)
ax2.set_title('Test Data')


for bar in bars2_sites:
    height = bar.get_height()
    ax2.annotate(f'{height}', xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3), textcoords="offset points", ha='center', fontsize=9, color='royalblue')


for bar in bars2_samples:
    height = bar.get_height()
    ax2b.annotate(f'{height}', xy=(bar.get_x() + bar.get_width() / 2, height),
                  xytext=(0, 3), textcoords="offset points", ha='center', fontsize=9, color='firebrick')

fig.suptitle('Sites and Samples per City (Train vs Test)', fontsize=16)

plt.tight_layout()
plt.subplots_adjust(top=0.88)
plt.savefig('../imgs/sites_samples_per_city.png')
plt.savefig('../imgs/sites_samples_per_city.pdf')
plt.show()


In [ ]:
# --- 1) Chuẩn hoá & đếm ---
def prep_counts(df, lat_col='site_latitude', lon_col='site_longitude', decimals=3):
    tmp = df.copy()
    tmp['lat'] = tmp[lat_col].round(decimals)
    tmp['lon'] = tmp[lon_col].round(decimals)
    out = tmp.groupby(['lat', 'lon'], as_index=False).size()
    out.rename(columns={'size': 'count'}, inplace=True)
    return out

train_counts = prep_counts(train)
test_counts  = prep_counts(test)

# --- 2) Scale kích thước dùng chung ---
max_count = max(train_counts['count'].max(), test_counts['count'].max())
size_scale = 800 / max_count  # chỉnh 800 cho hợp mắt

# --- 3) Vẽ: khác marker, có viền, zorder ---
plt.figure(figsize=(10, 20))
eps = 0.03  # 4) jitter nhẹ
plt.scatter(train_counts['lon']-eps, train_counts['lat']-eps,
            s=train_counts['count'] * size_scale, marker='o',
            c='tab:blue', alpha=0.6, label='Train',
            edgecolors='white', linewidth=0.7, zorder=2)

plt.scatter(test_counts['lon']+eps, test_counts['lat']+eps,
            s=test_counts['count'] * size_scale, marker='s',
            c='tab:green', alpha=0.6, label='Test',
            edgecolors='white', linewidth=0.7, zorder=3)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Bubble Plot of Measurement Sites (Train vs Test)')
plt.legend()
plt.grid(True, linestyle=':', alpha=0.6)
plt.gca().set_aspect('equal', adjustable='box')  # 5)
plt.tight_layout()
plt.savefig('../imgs/sites_per_location.png')
plt.savefig('../imgs/sites_per_location.pdf')
plt.show()


In [ ]:
prefixes = [
        "sulphurdioxide","carbonmonoxide","nitrogendioxide","formaldehyde",
        "uvaerosolindex","ozone","uvaerosollayerheight","cloud"
    ]
def plot_null_city_date(train,test,prefix):
    assert prefix in prefixes, "Invalid prefix"
    
    tr = train.copy()
    te = test.copy()
    tr['date'] = pd.to_datetime(tr['date'], errors='coerce')
    te['date'] = pd.to_datetime(te['date'], errors='coerce')
    tr = tr.dropna(subset=['date'])
    te = te.dropna(subset=['date'])
    tr_cities = pd.unique(tr['city'])
    te_cities = pd.unique(te['city'])
    
    # Map y (train/test) + jitter
    gap = 1
    te_map = {c: i for i, c in enumerate(te_cities, start=1)}                       # 1..4
    tr_map = {c: len(te_cities) + gap + i for i, c in enumerate(tr_cities, start=1)}# 6..9
    rng = np.random.default_rng(42)
    tr['_y'] = tr['city'].map(tr_map) + rng.uniform(-0.06, 0.06, len(tr))
    te['_y'] = te['city'].map(te_map) + rng.uniform(-0.06, 0.06, len(te))
    
    for col in train.columns:
        if col.startswith(prefix+"_"):
            check_null = col
            break
    tr_na = tr[tr[check_null].isna()]
    te_na = te[te[check_null].isna()]
    tr_full = tr[tr[check_null].notna()]
    te_full = te[te[check_null].notna()]
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.scatter(tr_na['date'], tr_na['_y'], s=18, alpha=0.8, label='Train-null', c='lightsteelblue')
    ax.scatter(te_na['date'], te_na['_y'], s=18, alpha=0.8, label='Test-null',  c='peachpuff')
    ax.scatter(tr_full['date'], tr_full['_y'], s=6, alpha=0.8, label='Train-full', c='tab:blue')
    ax.scatter(te_full['date'], te_full['_y'], s=6, alpha=0.8, label='Test-full', c='tab:orange')

    yticks  = list(te_map.values()) + list(tr_map.values())
    ylabels = list(te_cities)       + list(tr_cities)
    ax.set_yticks(yticks)
    ax.set_yticklabels(ylabels)
    ax.set_ylim(0.5, (len(te_cities) + len(tr_cities)) + 1.5 )
    ax.axhline(len(te_cities)+0.5, linestyle='--', alpha=0.3)
    
    loc = AutoDateLocator()
    ax.xaxis.set_major_locator(loc)
    ax.xaxis.set_major_formatter(ConciseDateFormatter(loc))
    
    ax.set_xlabel('Date')
    ax.set_ylabel('City')
    ax.set_title('City–Date scatter: Missing values for attribute "{}"'.format(prefix))
    ax.grid(axis='x', linestyle=':', alpha=0.6)
    ax.legend()
    plt.tight_layout()
    return fig, ax


In [ ]:
prefixes = [
        "sulphurdioxide","carbonmonoxide","nitrogendioxide","formaldehyde",
        "uvaerosolindex","ozone","uvaerosollayerheight","cloud"
    ]
def plot_null_city_date(train, test, prefix, ax=None, show_legend=True):
    assert prefix in prefixes, "Invalid prefix"
    
    tr = train.copy()
    te = test.copy()
    tr['date'] = pd.to_datetime(tr['date'], errors='coerce')
    te['date'] = pd.to_datetime(te['date'], errors='coerce')
    tr = tr.dropna(subset=['date'])
    te = te.dropna(subset=['date'])
    tr_cities = pd.unique(tr['city'])
    te_cities = pd.unique(te['city'])
    
    # Map y (train/test) + jitter
    gap = 1
    te_map = {c: i for i, c in enumerate(te_cities, start=1)}                       # 1..n
    tr_map = {c: len(te_cities) + gap + i for i, c in enumerate(tr_cities, start=1)}# ...
    rng = np.random.default_rng(42)
    tr['_y'] = tr['city'].map(tr_map) + rng.uniform(-0.06, 0.06, len(tr))
    te['_y'] = te['city'].map(te_map) + rng.uniform(-0.06, 0.06, len(te))

    for col in train.columns:
        if col.startswith(prefix + "_"):
            check_null = col
            break

    tr_na = tr[tr[check_null].isna()]
    te_na = te[te[check_null].isna()]
    tr_full = tr[tr[check_null].notna()]
    te_full = te[te[check_null].notna()]

    # dùng ax được truyền vào; nếu không có thì tự tạo
    if ax is None:
        fig, ax = plt.subplots(figsize=(12, 6))
    else:
        fig = ax.figure

    ax.scatter(tr_na['date'], tr_na['_y'], s=18, alpha=0.8, label='Train-null', c='lightsteelblue')
    ax.scatter(te_na['date'], te_na['_y'], s=18, alpha=0.8, label='Test-null',  c='peachpuff')
    ax.scatter(tr_full['date'], tr_full['_y'], s=6,  alpha=0.8, label='Train-full', c='tab:blue')
    ax.scatter(te_full['date'], te_full['_y'], s=6,  alpha=0.8, label='Test-full',  c='tab:orange')

    yticks  = list(te_map.values()) + list(tr_map.values())
    ylabels = list(te_cities)       + list(tr_cities)
    ax.set_yticks(yticks)
    ax.set_ylim(0.5, (len(te_cities) + len(tr_cities)) + 1.5 )
    
    # only show y on left hand side
    if ax.get_subplotspec().is_first_col():
        ax.set_yticklabels(ylabels)
    else:
        ax.set_yticklabels([])

    ax.axhline(len(te_cities)+0.5, linestyle='--', alpha=0.3)

    loc = AutoDateLocator()
    ax.xaxis.set_major_locator(loc)
    ax.xaxis.set_major_formatter(ConciseDateFormatter(loc))

    ax.set_xlabel('Date')
    if ax.get_subplotspec().is_first_col():
        ax.set_ylabel('City')
    ax.set_title(f'Nulls: "{prefix}"')

    ax.grid(axis='x', linestyle=':', alpha=0.6)
    if show_legend:
        ax.legend()

    return fig, ax


In [ ]:
nrows, ncols = 4, 2
fig, axs = plt.subplots(nrows, ncols, figsize=(18, 20), sharex=True, sharey=True)

handles = labels = None
for i, prefix in enumerate(prefixes):
    ax = axs.flat[i]
    fig, ax = plot_null_city_date(train, test, prefix, ax=ax, show_legend=True)
    if i == 0:
        handles, labels = ax.get_legend_handles_labels()

fig.suptitle('City–Date scatter: Missing values by attribute', y=0.995)
fig.tight_layout(rect=[0, 0, 1, 0.98])
plt.savefig('../imgs/missing_values_by_attribute.png')
plt.savefig('../imgs/missing_values_by_attribute.pdf')
plt.show()


In [ ]:
# full data

# transform date colums to datetime
tr = train.copy()
te = test.copy()
tr['date'] = pd.to_datetime(tr['date'], errors='coerce')
te['date'] = pd.to_datetime(te['date'], errors='coerce')
tr = tr.dropna(subset=['date'])
te = te.dropna(subset=['date'])

# get cities list
tr_cities = pd.unique(tr['city'])
te_cities = pd.unique(te['city'])

# Map y (train/test) + jitter
te_map = {c: i for i, c in enumerate(te_cities, start=1)}
tr_map = {c: len(te_cities) + i for i, c in enumerate(tr_cities, start=1)}
rng = np.random.default_rng(42)
tr['_y'] = tr['city'].map(tr_map) + rng.uniform(-0.06, 0.06, len(tr))
te['_y'] = te['city'].map(te_map) + rng.uniform(-0.06, 0.06, len(te))

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
ax.scatter(tr['date'], tr['_y'], s=18, alpha=0.8, label='Train', c='tab:blue')
ax.scatter(te['date'], te['_y'], s=18, alpha=0.8, label='Test',  c='tab:orange')

yticks  = list(te_map.values()) + list(tr_map.values())
ylabels = list(te_cities)       + list(tr_cities)
ax.set_yticks(yticks)
ax.set_yticklabels(ylabels)
ax.set_ylim(0.5, (len(te_cities) + len(tr_cities)) + 0.5)
ax.axhline(len(te_cities)+0.5, linestyle='--', alpha=0.3)

loc = AutoDateLocator()
ax.xaxis.set_major_locator(loc)
ax.xaxis.set_major_formatter(ConciseDateFormatter(loc))

ax.set_xlabel('Date')
ax.set_ylabel('City')
ax.set_title('Train (top) vs Test (bottom): City–Date scatter')
ax.grid(axis='x', linestyle=':', alpha=0.6)
ax.legend()
plt.tight_layout()
plt.savefig('../imgs/train_test_city_time.png')
plt.savefig('../imgs/train_test_city_time.pdf')
plt.show()



# Target variable analysis

In [ ]:
cities =  ['Kampala','Bujumbura','Lagos','Nairobi']
def hist_by_city(df,city):
    assert city in ['Kampala','Bujumbura','Lagos','Nairobi']
    y = df[df['city'] == city]['pm2_5']
    ys = np.linspace(y.min(), y.max(), 600)
    kde = gaussian_kde(y)
    return y, kde(ys)

In [ ]:
plt.figure(figsize=(12, 7))
for i in range(4):
    city = cities[i]
    y, kde_y = hist_by_city(train, city)
    plt.subplot(2, 2, i+1)
    plt.hist(y, bins=50, density=False, color='tab:green', rwidth=1.0, edgecolor='gray', linewidth=0.4, alpha=0.45, histtype='bar')
    plt.title(city)
    plt.xlabel('PM2.5 Value'); plt.ylabel('Count')
    plt.grid(axis='y', linestyle=':', alpha=0.6)
plt.tight_layout()
plt.savefig('../imgs/pm25_distribution_by_city.png')
plt.savefig('../imgs/pm25_distribution_by_city.pdf')
plt.suptitle('Distribution of PM2.5 Values by City', y=1.02)
plt.show()

In [ ]:
y = train['pm2_5'].to_numpy(float)
y = y[np.isfinite(y)]
y_pos = y[y > 0]

xs_lin = np.linspace(y.min(), y.max(), 600)
kde_lin = gaussian_kde(y)          # KDE /x (linear)

# KDE /log(x)
xs_log = np.logspace(np.log10(y_pos.min()), np.log10(y_pos.max()), 600)
z = np.log(y_pos)
kde_log = gaussian_kde(z)
pdf_logspace = kde_log(np.log(xs_log)) / xs_log

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ==== Left: linear ====
axes[0].hist(
    y, bins=50, density=True, color='tab:green',
    rwidth=1.0, edgecolor='gray', linewidth=0.4, alpha=0.45, histtype='bar'
)
axes[0].plot(xs_lin, kde_lin(xs_lin), linewidth=1, alpha=0.7, label='KDE')
axes[0].set_title('PM2.5 (linear x)')
axes[0].set_xlabel('PM2.5 Value'); axes[0].set_ylabel('Density')
axes[0].grid(axis='y', linestyle=':', alpha=0.6)
axes[0].legend()

# ==== Right: log ====
bins_log = np.logspace(np.log10(y_pos.min()), np.log10(y_pos.max()), 50)
axes[1].hist(
    y_pos, bins=bins_log, density=True, color='tab:green',
    rwidth=1.0, edgecolor='gray', linewidth=0.4, alpha=0.45, histtype='bar'
)
axes[1].set_xscale('log')
axes[1].plot(xs_log, kde_lin(xs_log), linewidth=1, alpha=0.7, label='KDE on x')
axes[1].plot(xs_log, pdf_logspace, linestyle='--', linewidth=1, alpha=0.9, label='KDE on log(x)')
axes[1].set_title('PM2.5 (log x)')
axes[1].set_xlabel('PM2.5 Value')
axes[1].grid(axis='y', which='both', linestyle=':', alpha=0.6)
axes[1].set_xlim(right= y_pos.max()*0.9)
axes[1].legend()

fig.suptitle('Distribution of PM2.5 Values in Training Data', y=0.98)
fig.tight_layout()

fig.savefig('../imgs/pm25_distribution.png', dpi=150)
fig.savefig('../imgs/pm25_distribution.pdf')
plt.show()


PM2.5 vs time

In [ ]:
df = train.copy()
df = df[np.isfinite(df['pm2_5'])]
df['month'] = pd.Categorical(df['month'].astype(int),
                             categories=range(1, 12+1), ordered=True)

sns.set_theme(style='whitegrid', context='notebook')

fig, ax = plt.subplots(figsize=(10, 6))

sns.boxplot(
    data=df, x='month', y='pm2_5',
    hue='month',     
    dodge=False,
    palette='Set2',
    width=0.6, linewidth=1.1,
    showfliers=True,       
    whis=(5, 95),
    ax=ax
)


ax.set_xlabel('Month')
ax.set_ylabel('PM2.5')
ax.set_title('PM2.5 by Month')

upper = np.nanpercentile(df['pm2_5'], 99)
ax.set_ylim(0, upper * 1.05)

ax.grid(axis='y', linestyle=':', alpha=0.6)
fig.tight_layout()
plt.legend('')

fig.savefig('../imgs/pm25_by_month_box.png')
fig.savefig('../imgs/pm25_by_month_box.pdf')
plt.show()


# Feature correlation

In [ ]:
not_selected_features = ['id', 'site_id', 'site_latitude', 'site_longitude', 'city', 'country', 'date', 'hour', 'month',
                         ]
feats = train.copy()
feats = feats.drop(columns=not_selected_features)
corr = feats.corr()
cols = corr.columns.tolist()
id_map = {col: i+1 for i, col in enumerate(cols)}         # {'very_long_name': 1, ...}

# Colname to ID because there are too many long names
corr_id = corr.copy()
corr_id.index   = [id_map[c] for c in cols]
corr_id.columns = [id_map[c] for c in cols]

# upper half mask
mask = np.triu(np.ones_like(corr_id, dtype=bool), k=1)

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(
    corr_id, mask=mask, cmap='coolwarm', center=0, square=True,
    cbar_kws={'shrink': 0.8}, linewidths=0.2, linecolor='white', ax=ax
)
ax.set_xlabel('Feature ID'); ax.set_ylabel('Feature ID')
ax.set_title('Feature Correlation (by ID)')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
plt.tight_layout()
plt.savefig('../imgs/feature_correlation_original.png')
plt.savefig('../imgs/feature_correlation_original.pdf')
plt.show()


In [ ]:
train[[col for col in train.columns if col.startswith('cloud_')]]

In [ ]:
def top_corr_pairs(corr, id_map, top_n=30, min_abs=0.8):
    mask = np.triu(np.ones(corr.shape, dtype=bool), k=1)
    df = corr.where(mask).stack().reset_index()
    df.columns = ['feature1', 'feature2', 'corr']
    df['abs_corr'] = df['corr'].abs()
    df['id1'] = df['feature1'].map(id_map)
    df['id2'] = df['feature2'].map(id_map)
    out = df[df['abs_corr'] >= min_abs].sort_values('abs_corr', ascending=False)
    if top_n is not None:
        out = out.head(top_n)
    return out[['id1','id2','corr','abs_corr','feature1','feature2']]

top_pairs = top_corr_pairs(corr, id_map, top_n=30, min_abs=0.7)
top_pairs

In [ ]:
def unify_with_majority(df, new_col='sensor_zenith_angle', decimals=3):
    src_cols = [c for c in df.columns if new_col in c and c != new_col]
    if not src_cols:
        return df

    # force numeric for row_majority function
    src_raw = df[src_cols].apply(pd.to_numeric, errors='coerce')
    src_rnd = src_raw.round(decimals)

    def row_majority(i):
        rnd = src_rnd.iloc[i].dropna()
        raw = src_raw.iloc[i].dropna()
        if rnd.empty:
            return np.nan

        cnt = Counter(rnd.values.tolist())
        maxc = max(cnt.values())
        # Take majority values
        tops = sorted([v for v, k in cnt.items() if k == maxc])
        chosen_r = tops[0]

        # median of original values corresponding to the chosen rounded value
        mask = (src_rnd.iloc[i] == chosen_r) & src_raw.iloc[i].notna()
        cand = src_raw.iloc[i][mask].values
        if cand.size:
            return float(np.median(cand))
        # fallback: first value
        return float(raw.iloc[0]) if not raw.empty else np.nan

    df[new_col] = [row_majority(i) for i in range(len(df))]

    # drop source columns
    df = df.drop(columns=src_cols)
    return df


In [ ]:
df = train.copy()
df = df.drop(columns=not_selected_features)
df = unify_with_majority(df, new_col='sensor_zenith_angle', decimals=3)
df = unify_with_majority(df, new_col='sensor_azimuth_angle', decimals=3)
df = unify_with_majority(df, new_col='solar_zenith_angle', decimals=3)
df = unify_with_majority(df, new_col='solar_azimuth_angle', decimals=3)
df = unify_with_majority(df, new_col='altitude', decimals=3)

In [ ]:
uva = ['uvaerosollayerheight_aerosol_height',
 'uvaerosollayerheight_aerosol_pressure',
 'uvaerosollayerheight_aerosol_optical_depth']
df = df.drop(columns=uva)

In [ ]:
corr = df.corr()
cols = corr.columns.tolist()
id_map = {col: i+1 for i, col in enumerate(cols)}         # {'very_long_name': 1, ...}

# Colname to ID because there are too many long names
corr_id = corr.copy()
corr_id.index   = [id_map[c] for c in cols]
corr_id.columns = [id_map[c] for c in cols]

# upper half mask
mask = np.triu(np.ones_like(corr_id, dtype=bool), k=1)

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(
    corr_id, mask=mask, cmap='coolwarm', center=0, square=True,
    cbar_kws={'shrink': 0.8}, linewidths=0.2, linecolor='white', ax=ax
)
ax.set_xlabel('Feature ID'); ax.set_ylabel('Feature ID')
ax.set_title('Feature Correlation (by ID)')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
plt.tight_layout()
# plt.savefig('../imgs/feature_correlation_new.png')
# plt.savefig('../imgs/feature_correlation_new.pdf')
plt.show()


Finally: Target variable correlation with others

In [ ]:
def top_with_target(corr, target='pm2_5', id_map=None, top_n=20):
    s = corr[target].drop(target).dropna()
    out = (pd.DataFrame({'feature': s.index, 'corr': s.values})
             .assign(abs_corr=lambda d: d['corr'].abs())
             .sort_values('abs_corr', ascending=False)
             .head(top_n))
    if id_map:
        out.insert(0, 'id', out['feature'].map(id_map))
    return out[['id','corr','abs_corr','feature']] if id_map else out

top_target = top_with_target(corr, target='pm2_5', id_map=id_map, top_n=20)
top_target